In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import random

In [ ]:
# Define the device to be used (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the transform to be applied to the images
transform = transforms.Compose(
    [transforms.RandomCrop(32, padding=4),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
# Define the training and testing datasets
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# Define the batch size and the number of workers for the data loaders
batch_size = 32
num_workers = 2


# Define the data loaders for the training and testing sets
trainloader = DataLoader(trainset, batch_size=batch_size,
                          shuffle=True, num_workers=num_workers)
testloader = DataLoader(testset, batch_size=batch_size,
                         shuffle=False, num_workers=num_workers)



In [ ]:

# Define a function to split the data into non-iid subsets
def non_iid_data_split(dataset, num_clients, num_shards_per_client):
    data_split = []
    idx_shard = np.arange(len(dataset)) % num_shards_per_client
    labels = dataset.targets
    idx_shard_labels = idx_shard * 10 + labels
    unique_labels = np.unique(idx_shard_labels)
    np.random.shuffle(unique_labels)
    num_labels_per_client = int(len(unique_labels) / num_clients)

    for i in range(num_clients):
        client_idxs = []
        for j in range(num_labels_per_client):
            label = unique_labels[i * num_labels_per_client + j]
            indices = np.where(idx_shard_labels == label)[0]
            np.random.shuffle(indices)
            client_idxs += list(indices)
        data_split.append(DatasetSubset(dataset, client_idxs))

    return data_split

In [ ]:
# Define the dataset subset class
class DatasetSubset(Dataset):
    def __init__(self, dataset, indices):
        self.dataset = dataset
        self.indices = indices

    def __getitem__(self, idx):
        return self.dataset[self.indices[idx]]

    def __len__(self):
        return len(self.indices)

In [ ]:
# Define the ResNet-18 model
class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        self.resnet18 = torchvision.models.resnet18(pretrained=True)
        self.fc = nn.Linear(1000, 10)

    def forward(self, x):
        x = self.resnet18(x)
        x = self.fc(x)
        return x

# Define the ResNet-18 model
model = torchvision.models.resnet18(pretrained=True)
# model.eval()
model = ResNet18().to(device)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the hyperparameters
num_clients = 10
num_epochs = 10
num_shards_per_client = 2
epsilon = 0.03

# Split the data into non-iid subsets for each client
data_splits = non_iid_data_split(trainset, num_clients, num_shards_per_client)

# Create the local models for each client
local_models = [ResNet18().to(device) for _ in range(num_clients)]

In [ ]:
# Define the optimizer and learning rate scheduler
optimizer = optim.SGD(local_models[0].parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60, 120, 160], gamma=0.2)

In [ ]:
# Train the model using federated learning
num_rounds = 10
for round in range(num_rounds):
    print(f"Round {round+1}:")

    # Select a subset of clients to participate in this round
    num_clients = 5
    client_idxs = np.random.choice(len(data_splits), num_clients, replace=False)

    # Initialize an empty list to store the gradients of the clients
    client_grads = []

    # Train the selected clients on their local data and accumulate their gradients
    for idx in client_idxs:
        # Load the client data and define the data loader
        client_data = data_splits[idx]
        client_loader = DataLoader(client_data, batch_size=batch_size, shuffle=True)

        # Define the client model and send it to the device
        client_model = ResNet18().to(device)

        # Define the client optimizer and train the model
        client_optimizer = optim.SGD(client_model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
        client_model.train()
        for epoch in range(num_epochs):
            client_loss = 0.0
            for batch_idx, (inputs, targets) in enumerate(client_loader):
                inputs, targets = inputs.to(device), targets.to(device)
                client_optimizer.zero_grad()

                # Perform adversarial attack
                inputs.requires_grad = True
                outputs = client_model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                inputs_grad = inputs.grad.detach()
                inputs.requires_grad = False
                epsilon = 0.03
                inputs_perturbed = inputs + epsilon*inputs_grad.sign()
                inputs_perturbed = torch.clamp(inputs_perturbed, 0, 1)

                # Update the model
                outputs = client_model(inputs_perturbed)
                loss = criterion(outputs, targets)
                loss.backward()
                client_optimizer.step()

                # Accumulate the gradients
                if batch_idx == len(client_loader)-1:
                    client_grads.append([param.grad.data.cpu().numpy().tolist() for param in client_model.parameters()])




    # Compute the average of the gradients of the selected clients
    avg_grads = []
    for i in range(len(client_grads[0])):
        avg_grad = torch.zeros_like(client_grads[0][i])
        for j in range(len(client_grads)):
            avg_grad += client_grads[j][i]
        avg_grad /= len(client_grads)
        avg_grads.append(avg_grad)

    # Perturb the average gradient with Gaussian noise
    for i in range(len(avg_grads)):
        avg_grads[i] += torch.randn_like(avg_grads[i]) * epsilon
    # Update the global model with the perturbed average gradient
    for param, avg_grad in zip(model.parameters(), avg_grads):
        param.data -= avg_grad.reshape(param.data.shape) * 0.1

Round 1:


In [ ]:
    # Evaluate the model on the test set
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)

            # Perform adversarial attack
            inputs.requires_grad = True
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            inputs_grad = inputs.grad.detach()
            inputs.requires_grad = False
            epsilon = 0.03
            inputs_perturbed = inputs + epsilon*inputs_grad.sign()
            inputs_perturbed = torch.clamp(inputs_perturbed, 0, 1)

            # Compute the statistics
            outputs = model(inputs_perturbed)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    print(f'Test Loss: {test_loss/(batch_idx+1):.3f} | Test Acc: {100*correct/total:.3f}')

# Save the model weights
torch.save(model.state_dict(), 'resnet18_cifar10.pth')


In [ ]:
##########################

In [ ]:
#########################

In [ ]:
#############

In [ ]:
#####################################################################################################################################################

In [ ]:
####################################################################################################################################################################################

In [ ]:
##################################################################################################################################

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
# Define the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the transformation for the data
transform = transforms.Compose(
    
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load the CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [2]:
# # Define the model architecture
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(3, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(16 * 5 * 5, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 10)

#     def forward(self, x):
#         x = self.pool(nn.functional.relu(self.conv1(x)))
#         x = self.pool(nn.functional.relu(self.conv2(x)))
#         x = x.view(-1, 16 * 5 * 5)
#         x = nn.functional.relu(self.fc1(x))
#         x = nn.functional.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x


# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
#         self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
#         self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
#         self.fc1 = nn.Linear(4 * 4 * 128, 256)
#         self.fc2 = nn.Linear(256, 10)
#         self.relu = nn.ReLU(inplace=True)
#         self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.softmax = nn.Softmax(dim=1)
        
#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.relu(x)
#         x = self.maxpool(x)
#         x = self.conv2(x)
#         x = self.relu(x)
#         x = self.maxpool(x)
#         x = self.conv3(x)
#         x = self.relu(x)
#         x = self.maxpool(x)
#         x = x.view(-1, 4 * 4 * 128)
#         x = self.fc1(x)
#         x = self.relu(x)
#         x = self.fc2(x)
#         x = self.softmax(x)
#         return x

# class Densenet(nn.Module):
#     def __init__(self):
#         super(Densenet, self).__init__()
#         self.densenet = torchvision.models.densenet121(pretrained=True)
#         self.densenet.classifier = nn.Linear(1024, 10)
#         self.softmax = nn.Softmax(dim=1)
        
#     def forward(self, x):
#         x = self.densenet(x)
#         x = self.softmax(x)
#         return x

class GoogLeNet(nn.Module):
    def __init__(self):
        super(GoogLeNet, self).__init__()
        self.googlenet = models.googlenet(pretrained=True)
        self.googlenet.fc = nn.Linear(1024, 10)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.googlenet(x)
        x = self.softmax(x)
        return x


In [3]:
def targeted_fgsm_attack(model, loss_fn, images, labels, epsilon, targeted_labels):
    images.requires_grad = True
    outputs = model(images)
    loss = loss_fn(outputs, labels)
    model.zero_grad()
    loss.backward()
    images_grad = images.grad.data
    perturbed_images = images + epsilon * torch.sign(images_grad)
    perturbed_images = torch.clamp(perturbed_images, 0, 1)
    perturbed_outputs = model(perturbed_images)
    perturbed_loss = loss_fn(perturbed_outputs, targeted_labels)
    return perturbed_loss, perturbed_images

def federated_averaging(models):
    new_state_dict = {}
    for key in models[0].state_dict().keys():
        new_state_dict[key] = torch.mean(torch.stack([model.state_dict()[key].float() for model in models]), dim=0)
    return new_state_dict


In [4]:
# def train(model, trainloader, optimizer, criterion, epsilon, targeted_labels):
#     model.train()
#     train_loss = 0
#     correct = 0
#     total = 0
#     for batch_idx, (inputs, targets) in enumerate(trainloader):
#         inputs, targets = inputs.to(device), targets.to(device)
        
#         # Convert the targeted labels tensor to float
#         repeated_targets = targeted_labels.float().repeat(inputs.size(0), 1)

#         # Generate adversarial examples using targeted FGSM
#         inputs.requires_grad = True
#         outputs = model(inputs)
#         loss = criterion(outputs, repeated_targets)
#         loss.backward()
#         perturbed_inputs = inputs + epsilon * torch.sign(inputs.grad)
#         perturbed_inputs = torch.clamp(perturbed_inputs, 0, 1)
#         inputs.requires_grad = False

#         # Train the model on the perturbed inputs
#         optimizer.zero_grad()
#         outputs = model(perturbed_inputs)
#         loss = criterion(outputs, targets)
#         loss.backward()
#         optimizer.step()

#         # Calculate training accuracy
#         train_loss += loss.item()
#         _, predicted = outputs.max(1)
#         total += targets.size(0)
#         correct += predicted.eq(targets).sum().item()

#     # Calculate and return training accuracy
#     train_acc = 100. * correct / total
#     return train_loss, train_acc


# def train(model, trainloader, optimizer, criterion, epsilon, targeted_labels):
#     model.train()
#     train_loss = 0
#     correct = 0
#     total = 0
#     for batch_idx, (inputs, targets) in enumerate(trainloader):
#         inputs, targets = inputs.to(device), targets.to(device)
        
#         # Convert the targeted labels tensor to float
#         repeated_targets = targeted_labels.float().repeat(inputs.size(0), 1)

#         # Generate adversarial examples using targeted FGSM
#         inputs.requires_grad = True
#         outputs = model(inputs)
#         loss = criterion(outputs, repeated_targets)
#         loss.backward()
#         perturbed_inputs = inputs + epsilon * torch.sign(inputs.grad)
#         perturbed_inputs = torch.clamp(perturbed_inputs, 0, 1)
#         inputs.requires_grad = False

#         # Train the model on the perturbed inputs
#         optimizer.zero_grad()
#         outputs = model(perturbed_inputs)
#         loss = criterion(outputs, targets)
#         loss.backward()
#         optimizer.step()

#         # Calculate training accuracy
#         train_loss += loss.item()
#         _, predicted = outputs.max(1)
#         total += targets.size(0)
#         correct += predicted.eq(targets).sum().item()

#     # Calculate and return training accuracy
#     train_acc = 100. * correct / total
#     return train_loss, train_acc


def train(model, trainloader, optimizer, criterion, epsilon, targeted_labels):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        # Convert the targeted labels tensor to float
        repeated_targets = targeted_labels.float().repeat(inputs.size(0), 1)

        # Generate adversarial examples using targeted FGSM
        perturbed_loss, perturbed_inputs = targeted_fgsm_attack(model, criterion, inputs, targets, epsilon, repeated_targets)
        perturbed_inputs = torch.clamp(perturbed_inputs, 0, 1)

        # Train the model on the perturbed inputs
        optimizer.zero_grad()
        outputs = model(perturbed_inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        # Calculate training accuracy
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    # Calculate and return training accuracy
    train_acc = 100. * correct / total
    return train_loss, train_acc


In [5]:
# Define the testing function
def test(model, testloader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # Calculate testing accuracy
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    # Calculate and return testing accuracy
    test_acc = 100. * correct / total
    return test_loss, test_acc

In [6]:
# Define the federated learning function
def federated_learning(models, trainloaders, testloader, optimizer, criterion, epsilon, targeted_labels, num_epochs):
    for epoch in range(num_epochs):
        # Train each model on their respective dataset
        train_losses = []
        train_accs = []
        for model, trainloader in zip(models, trainloaders):
            train_loss, train_acc = train(model, trainloader, optimizer, criterion, epsilon, targeted_labels)
            train_losses.append(train_loss)
            train_accs.append(train_acc)

        # Update the models by performing federated averaging
        new_state_dict = federated_averaging(models)
        for model in models:
            model.load_state_dict(new_state_dict)

        # Test the models on the testing dataset
        test_losses = []
        test_accs = []
        for model in models:
            test_loss, test_acc = test(model, testloader, criterion)
            test_losses.append(test_loss)
            test_accs.append(test_acc)

        # Print the training and testing losses and accuracies
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('Training Loss: {}'.format(sum(train_losses) / len(train_losses)))
        print('Training Accuracy: {}%'.format(sum(train_accs) / len(train_accs)))
        print('Testing Loss: {}'.format(sum(test_losses) / len(test_losses)))
        print('Testing Accuracy: {}%\n'.format(sum(test_accs) / len(test_accs)))

# Initialize the models and the datasets for the federated learning experiment
num_models = 5
models = [GoogLeNet().to(device) for i in range(num_models)]
trainloaders = [torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True) for i in range(num_models)]
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False)
optimizer = optim.SGD(models[0].parameters(), lr=0.1, momentum=0.9)
criterion = nn.CrossEntropyLoss()
epsilon = 0.03
targeted_labels = torch.tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0]).to(device)

# Run the federated learning experiment
num_epochs = 35
federated_learning(models, trainloaders, testloader, optimizer, criterion, epsilon, targeted_labels, num_epochs)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth


  0%|          | 0.00/49.7M [00:00<?, ?B/s]

KeyboardInterrupt: ignored